# Numpy高级应用

In [4]:
import numpy as np

## ndarray对象内部机理

### ndarray内部组成结构

NumPy的ndarray提供了一种将**同质数据块**(可以是连续或跨越的,稍后将详细讲解)解释为**多维数组**对象的方式。正如你之前所
看到的那样,数据类型(dtype)决定了数据的解释方式,比如浮点数、整数、布尔值等。

ndarray如此强大的部分原因是所有数组对象都是数据块的一个跨度视图(strided view)。你可能想知道数组视图arr[::2,::-1]不复制任
何数据的原因是什么。简单地说,ndarray不只是一块内存和一个dtype,它还有跨度信息,这使得数组能以各种步幅(step size)在内存中
移动。更准确地讲,ndarray内部由以下内容组成：
* 一个指向数组(一个系统内存块)的**指针**。
* **数据类型**或dtype。
* 一个表示数组**形状**(shape)的**元组**,例如,一个10×5的数组,其形状为(10,5)。
        np.ones((10, 5)).shape
        (10, 5)
* 一个**跨度元组**(stride),其中的整数指的是为了前进到当前维度下一个元素需要“跨过”的字节数,例如,一个典型的(C顺序,稍后将详细讲解)3×4×5的float64(8个字节)数组,其跨度为(160,40,8)。
        np.ones((3, 4, 5),dtype=np.float64).strides
        (160, 40, 8)
        
虽然NumPy用户很少会对数组的跨度信息感兴趣,但它们却是构建**非复制式数组视图**的重要因素。跨度甚至可以是**负数**,这样会使
数组在内存中**后向**移动,比如在切片obj[::-1]或obj[:,::-1]中就是这样的，下图很好的描述了ndarray的内部结构组成：

![ndarray的内部结构](https://raw.githubusercontent.com/NemoHoHaloAi/machine_learning/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/ndarray%E7%9A%84%E5%86%85%E9%83%A8%E7%BB%93%E6%9E%84.png)

### NumPy数据类型体系

你可能偶尔需要检查数组中所包含的是否是整数、浮点数、字符
串或Python对象。因为浮点数的种类很多,判断dtype是否属于某个大
类的工作非常繁琐。幸运的是,dtype都有一个**超类**(比如np.integer和
np.floating),它们可以跟**np.issubdtype**函数结合使用:

        ints = np.ones(10, dtype=np.uint16)
        floats = np.ones(10, dtype=np.float32)
        np.issubdtype(ints.dtype, np.integer)
        True
        np.issubdtype(floats.dtype, np.floating)
        True
        
调用dtype的**mro**方法即可查看其所有的父类:
        
        np.float64.mro()
        [numpy.float64,
        numpy.floating,
        numpy.inexact,
        numpy.number,
        numpy.generic,float,
        object]
        
大部分NumPy用户完全不需要了解这些知识,但是这些知识偶尔还是能派上用场的。下图说明了dtype体系以及父子类关系：
![dtype类体系结构](https://github.com/NemoHoHaloAi/machine_learning/blob/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/dtype%E7%B1%BB%E4%BD%93%E7%B3%BB%E7%BB%93%E6%9E%84.png?raw=true)

## 高级数组操作

除**花式索引**、**切片**、**布尔条件取子集**等操作之外,数组的操作方
式还有很多。虽然pandas中的高级函数可以处理数据分析工作中的许
多重型任务,但有时你还是需要编写一些在现有库中找不到的数据算
法。

### 数组重塑

鉴于我们已经学过的有关NumPy数组的知识,当你知道“无需复
制任何数据,数组就能从一个形状转换为另一个形状”时应该会感到
有一点吃惊。只需向数组的实例方法reshape传入一个表示新形状的元
组即可实现该目的。

#### 一维到多维 -- reshape

In [5]:
arr = np.random.randn(8)
arr

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

In [6]:
arr.reshape(4,2)

array([[ 2.19968261, -2.51749262],
       [-2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107],
       [ 0.81324603,  0.60564975]])

In [7]:
(arr.reshape(4,2)).reshape(2,4)

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

In [8]:
arr.reshape(np.random.randn(2,4).shape)

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

In [10]:
arr.reshape(2,-1) # -1,它表示该维度的大小由数据本身推断而来

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

#### 多维到一维 -- ravel,flatten

In [11]:
arr.reshape(2,4).ravel()

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

In [12]:
arr.reshape(2,4).flatten()

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

### C和Fortran顺序

与其他科学计算环境相反(如R和MATLAB),NumPy允许你更为灵活地控制数据在内存中的布局。默认情况下,NumPy数组是按行优先顺序创建的。在空间方面,这就意味着,对于一个二维数组,**每行**中的数据项是被存放在**相邻内存位置**上的。另一种顺序是列优先顺序,它意味着(猜到了吧)每列中的数据项是被存放在相邻内存位置上的。

由于一些历史原因,行和列优先顺序又分别称为C和Fortran顺序。在FORTRAN 77中(前辈们的语言),矩阵全都是列优先的。

像reshape和reval这样的函数,都可以接受一个表示数组数据存放顺序的**order**参数。一般可以是'**C**'或'**F**'(还有'A'和'K'等不常用的选项,具体请参考NumPy的文档)，下图展示了数组顺序：
![数组顺序](https://github.com/NemoHoHaloAi/machine_learning/blob/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/%E6%95%B0%E7%BB%84%E9%A1%BA%E5%BA%8F.png?raw=true)

In [16]:
arr.reshape(2,4)

array([[ 2.19968261, -2.51749262, -2.04008776, -0.60886891],
       [ 1.78442242,  0.90907107,  0.81324603,  0.60564975]])

In [14]:
arr.reshape(2,4).ravel(order='C')

array([ 2.19968261, -2.51749262, -2.04008776, -0.60886891,  1.78442242,
        0.90907107,  0.81324603,  0.60564975])

In [15]:
arr.reshape(2,4).ravel(order='F')

array([ 2.19968261,  1.78442242, -2.51749262,  0.90907107, -2.04008776,
        0.81324603, -0.60886891,  0.60564975])

二维或更高维数组的重塑过程比较令人费解。C和Fortran顺序的关键区别就是维度的行进顺序:
* C/行优先顺序:先经过更高的维度(例如,轴1会先于轴0被处理)。
* Fortran/列优先顺序:后经过更高的维度(例如,轴0会先于轴1被处理)。

### 数组的合并和拆分

In [17]:
arr1 = np.array([[1,2,3],[4,5,6]])
arr2 = np.array([[7,8,9],[10,11,12]])

#### numpy.concatenate

In [19]:
np.concatenate([arr1,arr2])

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [20]:
np.concatenate([arr1,arr2],axis=1)

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

#### numpy.vstack -- 相当于numpy.concatenate()

In [21]:
np.vstack([arr1,arr2])

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

#### numpy.hstack -- 相当于numpy.concatenate(axis=1)

In [22]:
np.hstack([arr1,arr2])

array([[ 1,  2,  3,  7,  8,  9],
       [ 4,  5,  6, 10, 11, 12]])

#### numpy.split -- vsplit,hsplit同理

In [31]:
np.vstack([arr1,arr2])

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [33]:
np.split(np.vstack([arr1,arr2]),indices_or_sections=[2,3])

[array([[1, 2, 3],
        [4, 5, 6]]), array([[7, 8, 9]]), array([[10, 11, 12]])]

In [37]:
np.split(np.vstack([arr1,arr2]),indices_or_sections=[4,1],axis=1)

[array([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]]), array([], shape=(4, 0), dtype=int64), array([[ 2,  3],
        [ 5,  6],
        [ 8,  9],
        [11, 12]])]

#### 数组连接拆分函数

* concatenate：最一般化的连接，沿一条轴连接一组数组；
* vstack、row_stack：以面向行的方式对一组数组进行堆叠；
* hstack：以面向列的方式对一组数组进行拼接；
* column_stack：类似于hstack，但会先将一维数组转换为二维列向量；
* dstack：以面向深度的方式对一组数组进行堆叠；
* split：沿指定轴在指定位置切分数组；
* hsplit、vsplit、dsplit：split的便捷方式，分别沿0、1、2进行拆分；

### 堆叠辅助类:r_和c_

### 元素的重复操作:tile和repeat

注意: 跟其他流行的数组编程语言(如MATLAB)不同,NumPy中
很少需要对数组进行重复(replicate) 译注4 。这主要是因为广播
(broadcasting,我们将在下一节中讲解该技术)能更好地满足该需求。

对数组进行重复以产生更大数组的工具主要是repeat和tile这两
个函数。

#### repeat会将数组中的各个元素重复一定次数,从而产生一个更大的数组

In [38]:
arr = np.arange(3)

##### 传入一个整数 -- 表示所有元素均重复这么多次

In [39]:
arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

##### 传入一个列表 -- 表示每个元素重复次数不一致

In [40]:
arr.repeat([2,4,8])

array([0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2])

##### 针对多维数组 -- 可以让它们的元素沿指定轴重复

In [44]:
arr_n = np.arange(12).reshape(4,3)

In [48]:
arr_n.repeat(2) # 没有设置轴向时，数据会被扁平化

array([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,
        8,  9,  9, 10, 10, 11, 11])

In [45]:
arr_n.repeat(2,axis=0)

array([[ 0,  1,  2],
       [ 0,  1,  2],
       [ 3,  4,  5],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [ 9, 10, 11]])

In [47]:
arr_n.repeat(2,axis=1)

array([[ 0,  0,  1,  1,  2,  2],
       [ 3,  3,  4,  4,  5,  5],
       [ 6,  6,  7,  7,  8,  8],
       [ 9,  9, 10, 10, 11, 11]])

#### np.tile

In [51]:
arr_n = np.arange(4).reshape(2,2)

#### 传入整数

In [52]:
np.tile(arr_n,2)

array([[0, 1, 0, 1],
       [2, 3, 2, 3]])

#### 传入列表

In [54]:
np.tile(arr_n,[3,2]) # 行向堆叠3遍，列向堆叠2遍

array([[0, 1, 0, 1],
       [2, 3, 2, 3],
       [0, 1, 0, 1],
       [2, 3, 2, 3],
       [0, 1, 0, 1],
       [2, 3, 2, 3]])

### 花式索引的等价函数：take和put

In [61]:
data = np.arange(16) * 10
data

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150])

#### 使用数组花式索引

In [62]:
data[[5,3,8,1]]

array([50, 30, 80, 10])

#### take -- 获取某个轴向上的元素

In [63]:
data.take([5,3,8,1])

array([50, 30, 80, 10])

#### put -- 设置某个轴向上的元素

In [64]:
data.put([5,3,8,1],-999)
data

array([   0, -999,   20, -999,   40, -999,   60,   70, -999,   90,  100,
        110,  120,  130,  140,  150])

#### 其他轴向 -- 注意：put是不接收axis参数的，只能在扁平一维上设置元素

In [66]:
data.reshape(4,4)

array([[   0, -999,   20, -999],
       [  40, -999,   60,   70],
       [-999,   90,  100,  110],
       [ 120,  130,  140,  150]])

In [70]:
data.reshape(4,4).take([1,2]) # 不设置轴向，默认是按照扁平化索引的，即线性的角标

array([-999,   20])

In [71]:
data.reshape(4,4).take([1,2],axis=0) # 设置横向，则表示索引的是行，而不是单个元素，axis=1同理

array([[  40, -999,   60,   70],
       [-999,   90,  100,  110]])

## 广播

广播(broadcasting)指的是不同形状的数组之间的算术运算的执
行方式。它是一种非常强大的功能,但也容易令人误解,即使是经验丰
富的老手也是如此。

### 最简单广播 -- 数组乘以标量

In [72]:
arr = np.array([1,2,3,4,5])
arr * 4

array([ 4,  8, 12, 16, 20])

### 距平化处理

In [73]:
arr - arr.mean()

array([-2., -1.,  0.,  1.,  2.])

### 低维度的值被广播到数组的任意维度

![低维度的值被广播到数组的任意维度](https://github.com/NemoHoHaloAi/machine_learning/blob/master/python%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90/images/%E4%BD%8E%E7%BB%B4%E5%BA%A6%E7%9A%84%E5%80%BC%E8%A2%AB%E5%B9%BF%E6%92%AD%E5%88%B0%E4%BB%BB%E6%84%8F%E7%BB%B4%E5%BA%A6.png?raw=true)

## ufunc高级应用

## 结构化和记录式数组

## 更多有关排序的话题

## Numpy的Matrix类

## 高级数组输入输出

## 性能建议